In [74]:
# """Character level RNN Model"""

In [2]:
import numpy as np

In [1]:
data = open('data/The_Fault_In_Our_Stars _John_Green.txt', 'r').read()
#train_data, test_data = data[:60000], data[60000:65821]

In [3]:
data = "This is the sample text"
chars = list(set(data))
chars

['h', ' ', 'p', 'e', 'l', 'x', 't', 's', 'a', 'T', 'm', 'i']

In [4]:
data_size, vocab_size = len(data), len(chars)
(data_size, vocab_size)

(23, 12)

In [5]:
char_to_ix = {ch:i for i, ch in enumerate(chars)}
char_to_ix

{' ': 1,
 'T': 9,
 'a': 8,
 'e': 3,
 'h': 0,
 'i': 11,
 'l': 4,
 'm': 10,
 'p': 2,
 's': 7,
 't': 6,
 'x': 5}

In [6]:
ix_to_char = {i:ch for i, ch in enumerate(chars)}
ix_to_char

{0: 'h',
 1: ' ',
 2: 'p',
 3: 'e',
 4: 'l',
 5: 'x',
 6: 't',
 7: 's',
 8: 'a',
 9: 'T',
 10: 'm',
 11: 'i'}

In [7]:
# hyper parameter
hidden_size = 100 # size of hidden layer of neurons
seq_length = 10  #25 number of steps to unroll the RNN for 
learning_rate = 1e-1

In [9]:
# model parameters
Wxh = np.random.rand(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.rand(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.rand(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [10]:
print(Wxh.shape)
print(Whh.shape)
print(Why.shape)
print(bh.shape)
print(by.shape)

(100, 12)
(100, 100)
(12, 100)
(100, 1)
(12, 1)


In [13]:
def loss_fun(inputs, targets, hprev):
    ''' 
    inputs, targets are both list of inetgers
    hprev is Hx1 array of initial hidden state
    returns the loss, the gradients on model parameters, the last hidden state
    '''
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forawrd pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1)) # one hot encoding..
        xs[t][inputs[t]] = 1 # ..one hot encoding
        
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by #unnormalised log probabilities for next chars
        ps[t] = np.exp(ys[t]/np.sum(np.exp(ys[t]))) # probabilities for next chars
        
        loss += -np.log(ps[t][targets[t], 0]) # softmax (cross entropy loss)
        
    # backward pass
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y.
        dWhy += np.dot(dy, hs[t].T)
        dby += by
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1-hs[t]*hs[t]) * dh # backprop through tanh non-linearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [14]:
def sample(h, seed_ix, n):
    '''
    Sample a sequence of integers from the model
    h is the model state, seed_ix is the seed letter for for time step
    '''
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y)/np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p = p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
        return ixes

In [15]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variable for adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

In [ ]:
while True:
    # prepare inputs (we are sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= len(data) or n ==0:
        hprev = np.zeros((hidden_size, 1)) # reset rnn memeory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p : p + seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1 : p + seq_length + 1]]
    
    # Sample from the model time to time
    #if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('%s' % (txt,), sep='', end='')
    
    # forward seq_length characters through the net and fetch gradients
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = loss_fun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    
    if n % 100 == 0: 
        print('\niter %d, loss: %f\n' % (n, smooth_loss)) # print progress
    
    # perform parameters update with adagrad
    for param, dparam, mem in zip(
                                    [Wxh, Whh, Why, bh, by],
                                    [dWxh, dWhh, dWhy, dbh, dby],
                                    [mWxh, mWhh, mWhy, mbh, mby]
                                    ):
        mem += dparam * dparam
        param = -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
        
    p += seq_length # move data pointer
    n += 1 # iteration counter

i
iter 0, loss: 24.824214

xmahllie htTs  ltha tpheisxxxmlhT aaapmethpxaapllTh iTlmpmxTTlthltiptp ehaellleiietipilx p aelTmpiia
iter 100, loss: 22.460381

ixeh lhpTtst httmsap metiilexeehTTixxmhhlplheestl tTspxpxhamaxelxeehesmaTtalalteix hxteTia e txam ti
iter 200, loss: 20.321604

mxepmmixi  apTiT  ii hlessiaieas ixmim ahehx seTsixxtapls Taemapllplsa xpxxt empt  xmlpehslxaseipxaa
iter 300, loss: 18.386456

xhhh i ptaThplsttTtasmtpeTmsemhpm s xttx lxh eh Tximsa ttthtxtpThtpims ipslpehteslialmaasp papmsTTl 
iter 400, loss: 16.635548

T limetattlmsTixlxpipieTTht xTmTtat pmlpxteh pp TeehTipslelieplhxTtpiTmashe ssp pxptpmTlexpesai  h i
iter 500, loss: 15.051341

xthislhaa tTTisesmahtsTtxTxxhpplxhem Tlilexiahtm axehlphhmatT pTtTmieilsx sapmmmmihspxllpe paeltepe 
iter 600, loss: 13.617963

ia Te iiepsmtptppe eepmhl  istxt Tmlmata slT aTxals pxemxmiaiThxmT aisTsahie xtt xahetmmeT xaaipmxx 
iter 700, loss: 12.321054

htp ahhheaxxTit aeTTxleTei sppmxaahlmsxathpli ehximatsmsmexpmlxaTml haltsTlps

imlmlilemalaliTsiimm a hhaites TTThax xhhTitslts leTsphehi xshpsTsxsexmTele mlelhxsxlemTTmiihtamTm h
iter 6500, loss: 0.033296

TpiTllsexpiips pimTamhseiphiT ax sixhaaxtix itlaei xaxi ti ailxpmmpe ma iTpaxphattixsls mhslameTipTe
iter 6600, loss: 0.029753

ppeis t lsheliht Tmmi tlT sisaaTTspthhixis phxlailThhxpia m  Tipptlaaet ialemiie pihaaxps mm mxalmlt
iter 6700, loss: 0.026548

mhx mpi lTexhxseimxhss  ixiaaxtplsssptiiplmellsTxaTxetTpmtTmpTT mpexm eamxaspatixehesaaeaTtsltpe i t
iter 6800, loss: 0.023648

ihlsappTlTlt sTehaxlaltx hxphxtah xexehsxxTpehmsxtThTap le haipeiaxTlsxtsasTsipa Tmtatpeaphesihmlx p
iter 6900, loss: 0.021024

lsmmhmxxitxetsp TmhTexamaaa mphiap exTpmel lstphxhxlaseehiatpi laTsetT extTTtpliTTs sihias xlxTpstlt
iter 7000, loss: 0.018650

stamlmhll TpiTTham haaamstpT aah xxxslalit sTlteehiplxhhlThTpsetlxsp lTslmlaeatexplihxihhlllllahips 
iter 7100, loss: 0.016502

a llitmspTplhp iaapp attsastats memmslsxhehxeiTTsxx ttT eeptptTpxixmxiTi eTtltmeptmetsseTpmaseTtssax
ite

hmtmxlipataslhxlTtxsexlt texlThTm lsTmittssTaxaiT lp sihiis phhT taaxaxemp pesxx iisxilxstpxhxmTm mp
iter 12900, loss: -0.003849

s s ltaeTheei leatt plsemastetTxiixpih msemTmppTelTiti iaiTxhtlpexapmm mtTllhem x Thexet  Tx xTxisax
iter 13000, loss: -0.003855

pxhhxlaismieialap iemTTleimxh Tpttiptpxxsahpalhmaaimha  ispstTmilhh ttsT xaeeiihsahsmmTlxxahp exTsei
iter 13100, loss: -0.003860

xmTtxphmhlaTTxlehhpliTimlxTthheta thTamx mltttellaesape taeTxmmplmmmtxthhtTatTtTTaiem t ax lsThihlTt
iter 13200, loss: -0.003865

ishhimhmhah  p hhsmtpxihmmxhhpptpxxTtleshllhhxThslle     aaatheiTtexalhxptaemelmxlppxhhahTslxleaptex
iter 13300, loss: -0.003869

 tmi lxl atxexseitsessemle e tsThshaiithxtiTxhmpamhs TipttatpapamTaiplshahaxpx ippphmtttelitahxissta
iter 13400, loss: -0.003873

mhl tmp hpTm txTli salxml  xhxleiiixpstaT pspaileTpThisTmaetittpththllxmsmlahpehslip asm epe hlTmtpx
iter 13500, loss: -0.003877

hixeii Tsmeieiixielppsmt sstThxlsptmlsl hiieleilxaeelhThaTpppp xaetxt a xphTmesa ii ll xxl

m xT xi xtipTamixappieaxms mpphTphaelTaT  ms mpamhlpihiepaislTxTTesTax lTT Tisielmstex lxliisexpxlxl
iter 25500, loss: -0.003911

am TTpihptip tsmehlaxmxsmhle apallxppTsphTl mtxmtsixT paal llh slalaaaatx ltlimie xttpexTaix  epltTm
iter 25600, loss: -0.003911

xixesa mlssllt axxmaamTlhi issxmhpaahsiTlxiTllelpleplTT hTTtxitmxtl  ep e lpa mllexstlipes ilTiiti l
iter 25700, loss: -0.003911

eTtaTptih leiixhhl eleaiapati lpmmTixTpteshTmsepiixmpalitxeTste hltmxepmhs TpimsTasaseeTsm lxsteseet
iter 25800, loss: -0.003911

ttithe a ihlmiehsTmxmeiipTpse  exTlalltaem aitlhaesmssTahplpiis tlihmttpmmxspTmTmssTthhasTtaitlsT at
iter 25900, loss: -0.003911

Tx apxxtTeia tphhmhihTas t TapaspapapattaTaxilhhptaehxplTTxTxp sT lp lx sahsxTet emtlishelemhT pmppl
iter 26000, loss: -0.003911

xhaihmxmtTTle ltTm mhesmT shspmlpiTxxeshexaixiiTeTphiT taptxexmtmaaThxpappi etpshpe thlatmatshaaeTse
iter 26100, loss: -0.003911

imspaae sstpxehTiTthti x txTtT  tleeTxstle sl TTxtpmhTtxxxtTsxsemTtxTptpxmThmpleeTxelleise

maetheltpTax masTtemi tahxthmiait  phsmmxtmppl ehtmhelaehamTTT essxxptslseTT TxmiltsealeTTpppxatm  p
iter 38100, loss: -0.003911

eeip xemeeetllahstTtthTta TTT xitsxssTpssm sesm iheph ttthxepmtasemtaxemtmepTmTaaisThlsxmmtaipe asap
iter 38200, loss: -0.003911

xiexhphilpeaeseshphTa htTtllhmpastitaaTpeTxaxapslhm sp ahlllh xi eTttxl hthsp msTa ipTsTtpms saeheps
iter 38300, loss: -0.003911

pmataixshmm xshx ltTmtaiaxsml Tx mximtthxha xai hhptpehteplmmpmesmahmxe liahe eapThph tTsmsmxlmxim t
iter 38400, loss: -0.003911

ptmlelt mThs  xatlTT xhlaspTash exlsixex mpxT tpphTampt slaheiaemlaTp memi sppimeeampil plTtisxhx a 
iter 38500, loss: -0.003911

iitTxhe TiTapi xThpTlhtlhixshia mpeaTaatxs eemtmsiT pimisilseTsTixtTixh mlplpTe lxam  ishxhx tas mtT
iter 38600, loss: -0.003911

esehiilTTpe t aTem ihsalThi x ihlTlimTxeppmhtpeipTTmpetaxxaillsmttlsimteteitmal pTmameeimmiseilTppel
iter 38700, loss: -0.003911

p stespesxlpTTtipahmss mesaaxxhmixma ahe ehepTxxxxss pslThxaxsps mlsmTppheapsltTt  tTaxml 

...

pesxhesetaxeasTtTs pxlpxipilTssheihxxmTiiTTelmtsx elheleTeilhhtthapltmpmhhpehlllTtapsThThmxltslttsle
iter 557300, loss: -0.003911

ellaamesmtaxiTittmxe paahis ihThihsli hm xieipxT piTehxmTesasmTse xalexiptstxtTilsshhTlxxllpit iei a
iter 557400, loss: -0.003911

mxspp hitailthitlmxpmsmtixT tll ti hehlxih heshsTtTatTaTh pelTistst xmT h tlaahesthalsmlpihaTTixlhtp
iter 557500, loss: -0.003911

ai Txl mieTstTppTs iat tlptehptsha eepstTTiTpxatmxehp txhihtTaa  tht tTlmam Tamslelxpmpxisemetssathm
iter 557600, loss: -0.003911

xeahhaltphh til TxhTm xisi t hTsllhthttTp Tp hmt Tm hiamTxiTsi lxeeesmxthaltppieax hhemtp ThTxetmeis
iter 557700, loss: -0.003911

a tsspslmhlxhhTThhax aaxelx ahhehpeailsepllslimpismsplThx xetslism meelT spltlpisphheTmhxellahtxpemx
iter 557800, loss: -0.003911

T  hpps ippt tmTtasha ihh imhipimhmxlTaapthTxhap tpT Tttlelasextemmihhplemx xTxshTxi aTtesx thax hTs
iter 557900, loss: -0.003911

hahls Txlh mlthaimhTxl elaeh lm plsamhTTtleiTxiT iip i eep pTTpTiexmplTemspxih TssT

 xxT h T phxp ieeTtsaaTmthapahpmh txe ataaT xxampiTaTmlaxxiphsxpaTTTheitsh haTlxaesTmalelsiTlpeameht
iter 569800, loss: -0.003911

hxlpm lTtlptlsipstTeTlehax eeTtTes itxTatpxtTpTlhehx iamshitTeihalTtiiatlpah pTheaaeTem hTaxptmemsxs
iter 569900, loss: -0.003911

hsslelsslhT ltxTTlhiihaxaisxmtaxlpesttlxsapTeelmxmxmsm tesexal smhTexTmlT     ealxmisaTapsth msseeal
iter 570000, loss: -0.003911

etxtesseeTmxippiltl ii ihathlilm lxxihpmaThm xpahltlateetmatsei  ma epixppiepttmailTm lepsmxmTt paea
iter 570100, loss: -0.003911

apsxpx altxTmxsm s silhelTleTmlpemxetximT xmhepamattxihteTxmeteexilxsh pipmTplxhmseelp ieplpTiapets 
iter 570200, loss: -0.003911

xpTThsmse h   axeeh p aalmshe siaaxeTl sTpxmmxts TtpTpplpsxsphllxTmmailhiixm aseTmmeplsae imlsmaas p
iter 570300, loss: -0.003911

xlhTlltx slixeleeaehlamspitlxaesxTxT hxhttsehimxxalxTtishxxhxmp tsaTt  lxhiaihpmxxhxtiTpheame iielhl
iter 570400, loss: -0.003911

silempi steasTt psthpeTs h pteTxah mhhtTilmeimlpp plthe tTaThtTTeppppa tmxmptlhm hh

xtTtsammiiThxlmtaiilappmtaaTailtpptatxsT txae hx aex  eimetttemxhlThhTethpt sThtamam tipmtpe eiltxam
iter 576100, loss: -0.003911

ai ll aaia x txhxitteelt ettxassealhs tiTxllalxhlTahplTTmiilxpmasshxaiteapxaxamTThth Tlllx  teTaaphx
iter 576200, loss: -0.003911

tttxTepxmlah ieilpt hsme tTxmsTimhxsillptmhmi tplaixtpTTxiehlmetxTTTTatihlixheTttah lp lTsThpxeihmhi
iter 576300, loss: -0.003911

haxesllielppipthxmplTsttilxhpixlhm a   tameill iiptemalxshlTshtxhsixeltpTmphlaetmta xmhelmepmamhxshh
iter 576400, loss: -0.003911

iTxTTmmishltmi h pixmTxlmphxmmpislTsastislxmspe TpsseisiaeTimmilpxTa ahmahl eepalpl  Thh eillatphTTl
iter 576500, loss: -0.003911

me eTehlipasht sTiexmtxihehhhaTtxlx xxatTpemTmehT pTxxl paatastxaaiihhi  ipmexeessmsheaemspaiieepxeh
iter 576600, loss: -0.003911

hl aiatx smlxhetsalaTxhpahemlsmhtahTeelTaxsmpptlp Txi mxitm TTxp mtphTtmpeixsts smmxexm Te pimtiell 
iter 576700, loss: -0.003911

mletpteahpe aseh m pmtipplhtp lsxsexhpxe tmpplx h TlT psThp p lshlxletxhxxsasTtepsp

sspm txt pthphTheap TtTeli esih mpmiixmllis ehttstixexe impesp sithleeemhpi  paTTemsTimsT aeisilmmtp
iter 588500, loss: -0.003911

 TmtlixTx ixhshiemla sseaepTshxTTsTtie iaxlmmxTtmlxhepxpsmsme milpieTTxTlxlmtihimixsmTtmsiTTssthhamp
iter 588600, loss: -0.003911

ttximlx ximTxiTtmteesexhttpsaehltTlsT m tm xmmTimTieaexTtah mt mm pmTpTl tpaett Th plsps lppexaxplxi
iter 588700, loss: -0.003911

immxTxplpsaxi iilimmxlithmptatih  xheipmtltxx hpimhaiilhhha  px xa lTeshsTa he   xTap m eilaemlm hxe
iter 588800, loss: -0.003911

i xslxmTxTtm  ia lhlaati heix pasemaamhehpx Tisaelxaspaxmsp t lmms mhtelhlaetm ti aepTpxxmtpl hllths
iter 588900, loss: -0.003911

aelTplaixmTxaTTTThTelltlaix  ttsTsTiameTTtaaxpp  TTsxtxxliehmipTmmaxliTm heTsa pxiTxlaaltemmlhmasxse
iter 589000, loss: -0.003911

shitmxa llTismppaxettes  hsex teeshlisTtiipxslimTTesssTa mlexi xxeittpmt miximahsapmhhTpspttpl lmmeT
iter 589100, loss: -0.003911

slhtexlmxatTamte hTlmslxlteTetmhx xT iiTiepai aTTlxihmexlai  texpspap ms hlimelipeh

psslehhesTlhpeppTsi exal ittltilxTsh hmaimi aismllpls txslm Tl hslT TihesTtahl aemaisxhsalxixtsamixp
iter 600900, loss: -0.003911

heTpal TlpTxitTthil hiTmxetximhiimlsaTtpxlisipplmTeTxTspiTl xi tmieiaTxlsitelhstxTapihilasli ThiaT h
iter 601000, loss: -0.003911

hiemeipmllhpptimhhthTsishmsaela apexTxxhsTxleaT TTpiaxlseelahllxmlhtheeehtaxl hlshaTxpaixtiixssTspta
iter 601100, loss: -0.003911

im eehsxstpxThxliithais m iexh ptpTmhimmx lsmmhemipxtTsmlt sTxthmhpilslxTTaphaT t emxiTsahshtmmx iim
iter 601200, loss: -0.003911

tahla  imlxmxieasspsh ip miihmmxxixhlsixpmlhT Tmpsxlsmspmxexe  ae ismTexeTmsxiiethpTTphsa lx sxtpm h
iter 601300, loss: -0.003911

hatpTaimhleehmtii alahetx mistp emxl  sihxmaa thmT aspl iiTisxsmTeeplmhlm xail th  hppppt m psssxtsl
iter 601400, loss: -0.003911

pm exhil